## Sync client examples

More examples can be found [here](https://apache-ignite-binary-protocol-client.readthedocs.io/en/latest/readme.html#examples)

In [1]:
import pandas as pd

from pyignite import GenericObjectMeta, Client
from pyignite.datatypes import String, IntObject, DoubleObject
from pyignite.datatypes.prop_codes import PROP_NAME, PROP_SQL_SCHEMA, PROP_QUERY_ENTITIES
from pyignite.datatypes.cluster_state import ClusterState

In [2]:
CACHE_CONFIG = {
    PROP_NAME: 'sql_public_student',
    PROP_SQL_SCHEMA: 'public',
    PROP_QUERY_ENTITIES: [
        {
            'table_name': 'student',
            'key_field_name': 'sid',
            'key_type_name': 'java.lang.Long',
            'field_name_aliases': [],
            'query_fields': [
                {
                    'name': 'sid',
                    'type_name': 'java.lang.Long',
                    'is_key_field': True,
                    'is_notnull_constraint_field': True,
                },
                {
                    'name': 'name',
                    'type_name': 'java.lang.String',
                },
                {
                    'name': 'login',
                    'type_name': 'java.lang.String',
                },
                {
                    'name': 'age',
                    'type_name': 'java.lang.Integer',
                },
                {
                    'name': 'gpa',
                    'type_name': 'java.math.Double',
                },
            ],
            'query_indexes': [],
            'value_type_name': 'sql_public_student_type',
            'value_field_name': None,
        },
    ],
}

In [3]:
class Student(
    metaclass=GenericObjectMeta,
    type_name='sql_public_student_type',
    schema={
        'name': String,
        'login': String,
        'age': IntObject,
        'gpa': DoubleObject,
    }
):
    pass

In [4]:
students = [
    {'name': 'John Doe', 'login': 'john_doe', 'age': 18, 'gpa': 2.5},
    {'name': 'Jane Doe', 'login': 'jane_doe', 'age': 25, 'gpa': 3.5},
    {'name': 'Eva Lu Ator', 'login': 'eva_lisp', 'age': 45, 'gpa': 4.9}
]

In [5]:
client = Client(partition_aware=True)

### Cache operations example

In [17]:
with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
    cache = client.get_or_create_cache(CACHE_CONFIG)
    
    cache.put_all({i: Student(**st) for i, st in enumerate(students)})
    cache
    
    all_data = cache.get_all([i for i in range(len(students))])
    
    row_names = ['name', 'login', 'age', 'gpa', 'version']
    rows = []
    
    for k,v in all_data.items():
        row = [k]
        row += [getattr(v, name, None) for name in row_names]
        rows.append(row)

    df = pd.DataFrame(rows, columns=['sid'] + row_names)
df

,sid,name,login,age,gpa,version
0,0,John Doe,john_doe,18,2.5,1
1,1,Jane Doe,jane_doe,25,3.5,1
2,2,Eva Lu Ator,eva_lisp,45,4.9,1


### Scan query example

In [11]:
with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
    cache = client.get_or_create_cache(CACHE_CONFIG)
    
    row_names = ['name', 'login', 'age', 'gpa', 'version']
    rows = []
    
    with cache.scan() as cursor:
        for k,v in cursor:
            row = [k]
            row += [getattr(v, name, None) for name in row_names]
            rows.append(row)

    df = pd.DataFrame(rows, columns=['sid'] + row_names)
df

,sid,name,login,age,gpa,version
0,0,John Doe,john_doe,18,2.5,1
1,1,Jane Doe,jane_doe,25,3.5,1
2,2,Eva Lu Ator,eva_lisp,45,4.9,1


### Sql query example

In [12]:
with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
    cache = client.get_or_create_cache(CACHE_CONFIG)
    cache.put_all({i: Student(**st) for i, st in enumerate(students)})
    
            
    with client.sql('select * from student', include_field_names=True) as cursor:
        row_names = next(cursor)
        rows = list(cursor)
        df = pd.DataFrame(rows, columns=row_names)

df

,SID,NAME,LOGIN,AGE,GPA
0,0,John Doe,john_doe,18,2.5
1,1,Jane Doe,jane_doe,25,3.5
2,2,Eva Lu Ator,eva_lisp,45,4.9


### Cluster API (set_state) example

In [15]:
with client.connect([('127.0.0.1', 10800 + i) for i in range(3)]):
    cluster = client.get_cluster()
    cluster.set_state(ClusterState.ACTIVE)
    # cluster.set_state(ClusterState.INACTIVE)
    # cluster.set_state(ClusterState.ACTIVE_READ_ONLY)
    
    cache = client.get_or_create_cache(CACHE_CONFIG)
    student = cache.get(1)
    student.age = 30
    cache.put(1, student)
    print(cache.get(1))

Student(name='Jane Doe', login='jane_doe', age=30, gpa=3.5, version=1)
